In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader

In [2]:
import json
import urllib

url = "https://raw.githubusercontent.com/Chaoyuuu/Gather-Town-Datasets/master/datasets-real-room.json"
response = urllib.request.urlopen(url)
data = json.loads(response.read())
data_length = len(data)
input_data = torch.zeros((data_length, 17, 13, 10))


for i in range(data_length - 18):
    for j in range(len(data[i]["room"])):
        input_data[i][data[i]["room"][j]["label"]][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1
        if data[i]["room"][j]["orientation"] == 0:
            input_data[i][16][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = -1
        elif data[i]["room"][j]["orientation"] == 1:
            input_data[i][15][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = -1
        elif data[i]["room"][j]["orientation"] == 2:
            input_data[i][16][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1
        else:
            input_data[i][15][data[i]["room"][j]["y"]][data[i]["room"][j]["x"]] = 1



z_dim = 4
x_dim = 17*13*10
bs = 64
epoch_round = 20
print("datasets number = " + str(data_length))


train_loader = DataLoader(dataset=input_data, batch_size=bs, shuffle=True)

datasets number = 246


In [3]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, x_dim))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=x_dim, h_dim1= 512, h_dim2=256, z_dim=z_dim)
if torch.cuda.is_available():
    vae.cuda()

In [4]:
vae

VAE(
  (fc1): Linear(in_features=2210, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=4, bias=True)
  (fc32): Linear(in_features=256, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=2210, bias=True)
)

In [5]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, x_dim), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [6]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        print(list(data.size())) # [81, 17, 13, 10]
        
        recon_batch, mu, log_var = vae(data) #torch.float32
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [7]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [8]:
for epoch in range(1, epoch_round):
    train(epoch)
#     test()

[64, 17, 13, 10]
Train Epoch: 1 [0/246 (0%)]	Loss: 1535.265137
[64, 17, 13, 10]
[64, 17, 13, 10]
[54, 17, 13, 10]
====> Epoch: 1 Average loss: 1430.2245
[64, 17, 13, 10]
Train Epoch: 2 [0/246 (0%)]	Loss: 1165.892090
[64, 17, 13, 10]
[64, 17, 13, 10]
[54, 17, 13, 10]
====> Epoch: 2 Average loss: 896.6375
[64, 17, 13, 10]
Train Epoch: 3 [0/246 (0%)]	Loss: 413.064575
[64, 17, 13, 10]
[64, 17, 13, 10]
[54, 17, 13, 10]
====> Epoch: 3 Average loss: 240.0349
[64, 17, 13, 10]
Train Epoch: 4 [0/246 (0%)]	Loss: 95.640297
[64, 17, 13, 10]
[64, 17, 13, 10]
[54, 17, 13, 10]
====> Epoch: 4 Average loss: 98.7035
[64, 17, 13, 10]
Train Epoch: 5 [0/246 (0%)]	Loss: 68.399567
[64, 17, 13, 10]
[64, 17, 13, 10]
[54, 17, 13, 10]
====> Epoch: 5 Average loss: 34.9343
[64, 17, 13, 10]
Train Epoch: 6 [0/246 (0%)]	Loss: -49.400143
[64, 17, 13, 10]
[64, 17, 13, 10]
[54, 17, 13, 10]
====> Epoch: 6 Average loss: -55.5873
[64, 17, 13, 10]
Train Epoch: 7 [0/246 (0%)]	Loss: -105.834213
[64, 17, 13, 10]
[64, 17, 13, 10

In [21]:
with torch.no_grad():
    z = torch.randn(1, z_dim).cuda()
    sample = vae.decoder(z).cuda()
    
    torch.set_printoptions(profile="full")    
    data = sample[0]
    
    output_tensor = torch.zeros((17, 13, 10))
    
    # reshape sample(1*200) -> output_tensor(17, 13, 10)
    for i, batch in enumerate(sample):
        i = 0
        for depth in range(17):
            for col in range(13):
                for row in range(10):
                    #print("depth, col, row: " + str(row)+", " + str(col) +", ", str(depth))
                    data = batch[i]
                    i += 1
                    if data >= 0.15:
                        output_tensor[depth][col][row] = 1
                                             
    
    object_dict = {
        0: 'WB',
        1: 'PR',
        2: 'T2',
        3: 'TV',
        4: 'B4',
        5: 'PP',
        6: 'MB',
        7: 'CC',
        8: 'CS',
        9: 'T3',
        10: 'B2',
        11: 'LP',
        12: 'MP',
        13: 'LB',
        14: 'DC'
    }


    room = [['--']*10 for i in range(13)]
    for col in range(13):
        for row in range(10):
            object_duplicate_flage = 0
            for depth in range(15):
                data = output_tensor[depth][col][row]
                
                if data == 1:
                    object_duplicate_flage += 1
                    room[col][row] = object_dict[depth]
                    
            if object_duplicate_flage > 1:
                print(f"dup in {col}, {row}")
    
    for col in range(13):
        print(room[col])
        
        
    print(sample)
                    
    # print room with object
    # WB: Whiteboard
    # PR: Projector Screen
    # T2: Chippendale Table (2x3)
    # TV: TV (Flatscreen)
    # B4: Bookshelf (2x4)
    # PP: Potted Plant (Spikey)
    # MB: Mod Chair
    # CC: Captain’s Chair
    # CS: Chair (Simple)
    # T3: Chippendale Table (3x3)
    # B2: Bookshelf [Tall] (1x2)
    # LP: Laptop
    # MP: Microphone
    # LB: Lucky Bamboo
    # DC: Dining Chair (Square)

dup in 0, 0
dup in 0, 4
dup in 0, 9
dup in 7, 6
['LB', 'PP', '--', '--', 'TV', '--', 'B4', '--', 'PP', 'LB']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', 'CS', 'T3', '--', '--', 'CS', '--', '--']
['--', '--', 'DC', 'CS', '--', '--', '--', 'CS', '--', '--']
['--', '--', '--', 'CS', '--', '--', 'LP', 'CS', '--', '--']
['--', '--', 'DC', 'CS', '--', '--', '--', 'CS', '--', '--']
['--', '--', 'DC', 'CS', '--', '--', 'DC', 'CS', '--', '--']
['--', '--', '--', 'CS', '--', '--', '--', 'CS', '--', '--']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
tensor([[8.8145e-03, 7.4729e-02, 6.6582e-02, 1.2192e-01, 2.1033e-01, 1.0887e-01,
         5.3007e-02, 3.9349e-02, 2.2622e-02, 1.1929e-07, 1.3852e-07, 5.0939e-0